# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [88]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 7
ROUNDS = 7
OTHER_DATE = '2023-10-27'
_RESETS = True
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

# Get IQ data

In [89]:
SHOTS = int(1e4)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None
P_AMBIG = 0.1

IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
# IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, noise_list=NOISE_LIST)

# Decode using predecoder

In [91]:
import pymatching
import stim
import cpp_soft_info
import numpy as np

model = simulator.stim_circ.detector_error_model(decompose_errors=False)
matching = pymatching.Matching.from_detector_error_model(model)

In [92]:
_DETAILED = True

result_predecoder = cpp_soft_info.decode_time_nn_predecode_grid(model, IQ_data[:], ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, threshold=0.15, _ntnn_edges = not _RESETS)


print(f"num_errors: {result_predecoder.decode_result.num_errors}")
print(f"mean num removed edges: {np.mean(result_predecoder.nb_rm_edges)}")
print(f"num removed edges: {np.sum(result_predecoder.nb_rm_edges)}")

num_errors: 122
mean num removed edges: 0.0872
num removed edges: 872


In [93]:
for threshold in np.linspace(0, 0.4, 20):
    result_predecoder = cpp_soft_info.decode_time_nn_predecode_grid(model, IQ_data[:], ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                               simulator.processed_scaler_dict, _detailed=_DETAILED, threshold=threshold, _ntnn_edges = not _RESETS)
    print(f"threshold: {threshold}, num_errors: {result_predecoder.decode_result.num_errors}, mean num removed edges: {np.mean(result_predecoder.nb_rm_edges)}, num removed edges: {np.sum(result_predecoder.nb_rm_edges)}")

threshold: 0.0, num_errors: 259, mean num removed edges: 1.6781, num removed edges: 16781
threshold: 0.021052631578947368, num_errors: 132, mean num removed edges: 0.2667, num removed edges: 2667
threshold: 0.042105263157894736, num_errors: 120, mean num removed edges: 0.1534, num removed edges: 1534
threshold: 0.06315789473684211, num_errors: 121, mean num removed edges: 0.1275, num removed edges: 1275
threshold: 0.08421052631578947, num_errors: 123, mean num removed edges: 0.1131, num removed edges: 1131
threshold: 0.10526315789473684, num_errors: 122, mean num removed edges: 0.1021, num removed edges: 1021
threshold: 0.12631578947368421, num_errors: 122, mean num removed edges: 0.0934, num removed edges: 934
threshold: 0.14736842105263157, num_errors: 122, mean num removed edges: 0.0877, num removed edges: 877
threshold: 0.16842105263157894, num_errors: 121, mean num removed edges: 0.0823, num removed edges: 823
threshold: 0.18947368421052632, num_errors: 121, mean num removed edges

In [94]:
# Soft Grid info

result = cpp_soft_info.decode_IQ_fast(model, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, nb_intervals=-1)

print(f"num_errors: {result.num_errors}")

num_errors: 130


In [95]:
# Hard decoder
# p_meas = 0.02
p_meas = -1

result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(model, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = not _RESETS)

print(f"num_errors: {result_informed.num_errors}")

num_errors: 125
